<a href="https://colab.research.google.com/github/rituparnamaiti/fcsl.co/blob/main/23CD71P01_ASSIGNMENT3_PART2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
pip install transformers

In [24]:
pip install torch

In [25]:
!pip install transformers accelerate peft

In [26]:
import pandas as pd

# dataset_path = '/content/NLP_ass_test.tsv'  # Replace with your dataset path
# data = pd.read_csv(dataset_path)
# posts = data['post_column_name']


test_df = pd.read_csv('/content/NLP_ass_test.tsv', sep='\t', header=None)

In [27]:
test_df.columns = ['statement', 'category']

In [28]:
test_df

,statement,category
0,i dont think im getting my baby them white 9 h...,normal
1,laura loomer raped me while screaming at me in...,hatespeech
2,<user> what did the old lady do was she a nazi...,normal
3,as much as i appreciate a format of open discu...,normal
4,sex be so good a bitch be slow stroking and cr...,offensive
...,...,...
1919,it always women trying this shit like dingbat ...,offensive
1920,because women would never lie about being sexu...,offensive
1921,how is all that awesome muslim diversity going...,offensive
1922,well my dear lgbtq brothers and sisters i do n...,hatespeech


In [29]:
# Import necessary classes from transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Define the model name
model_name = "google/flan-t5-small"  # or "google/flan-t5-base"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [18]:
!pip install peft

In [20]:
from peft import LoraConfig

In [30]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)

In [33]:
!pip install peft

In [34]:
from peft import LoraConfig, get_peft_model

In [35]:
model = get_peft_model(model, lora_config)

In [36]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [37]:
from datasets import Dataset
dataset = Dataset.from_pandas(test_df)


# Define the tokenization function
def tokenize_fn(examples):
    inputs = tokenizer(examples["statement"], padding="max_length", truncation=True)
    outputs = tokenizer(examples["category"], padding="max_length", truncation=True)
    inputs['labels'] = outputs['input_ids']
    return inputs

# Split into train and validation datasets (e.g., 80-20 split)
train_data, val_data = dataset.train_test_split(test_size=0.2).values()

# Tokenize train and validation data
train_tokenized = train_data.map(tokenize_fn, batched=True)
val_tokenized = val_data.map(tokenize_fn, batched=True)


Map:   0%|          | 0/1539 [00:00<?, ? examples/s]

Map:   0%|          | 0/385 [00:00<?, ? examples/s]

In [40]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    run_name="7e0df72cb3a9a6c2655a4fbab1a0cf1ecb9fa6e9",  # Specify a unique run name here
)

In [42]:
from transformers import Trainer
# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
)

In [43]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [44]:
from transformers import AutoTokenizer

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

def tokenize_fn(examples):
    inputs = tokenizer(examples["statement"], padding="max_length", truncation=True, return_tensors="pt")
    outputs = tokenizer(examples["category"], padding="max_length", truncation=True, return_tensors="pt")
    inputs['labels'] = outputs['input_ids']
    return inputs

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [45]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [46]:
print(train_tokenized[0])

{'statement': 'me quietly sitting in class with my friend and showing off my shoes to her look my friend your gay is showing', 'category': 'offensive', 'input_ids': [140, 19965, 3823, 16, 853, 28, 82, 1565, 11, 2924, 326, 82, 4439, 12, 160, 320, 82, 1565, 39, 16998, 19, 2924, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [47]:
print(len(train_tokenized))
print(len(val_tokenized))

1539
385


In [48]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
)


In [49]:
print("Train dataset:", train_tokenized)
print("Validation dataset:", val_tokenized)

Train dataset: Dataset({
    features: ['statement', 'category', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1539
})
Validation dataset: Dataset({
    features: ['statement', 'category', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 385
})


In [50]:
trainer.save_model("fine_tuned_flan_t5")

In [58]:
# Iterate over the tokenized validation data
for example in val_tokenized:
    # Prepare inputs for the model
    inputs = {}
    for key, val in example.items():
        if key != 'labels':
            # Check if val is already a tensor or a list of integers
            if isinstance(val, torch.Tensor):
                inputs[key] = val.unsqueeze(0)  # Add batch dimension
            elif isinstance(val, list) and all(isinstance(i, int) for i in val):
                inputs[key] = torch.tensor(val).unsqueeze(0)  # Convert to tensor and add batch dimension

    with torch.no_grad():
        outputs = model.generate(**inputs)

    # Print input, prediction, and actual values
    print("Input:", tokenizer.decode(inputs["input_ids"][0], skip_special_tokens=True))
    print("Prediction:", tokenizer.decode(outputs[0], skip_special_tokens=True))
    print("Actual:", tokenizer.decode(example["labels"], skip_special_tokens=True))
    print("-" * 50)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Input: user> he a fucking faggy kike oh yeah bill mitchell got it as well he turned on the alt right yesterday as well saying white genocide and jq were conspiracy theories
Prediction: Bill Mitchell got it as well he turned on the alt right yesterday as well saying
Actual: offensive
--------------------------------------------------
Input: user> that the retarded democrats for you
Prediction: user> that the retarded democrats for you
Actual: offensive
--------------------------------------------------
Input: couple friends you know bitches get funny
Prediction: a bitch is a bitch
Actual: normal
--------------------------------------------------
Input: muslim migrant rapes more women than canadian bernado yet media stays silent we wonder why no one trusts the media any more
Prediction: muslim migrant rapes more women than canadian bernado
Actual: offensive
--------------------------------------------------
Input: matt hale sits in prison due to being framed by an informant edgar steele 